Example of genetic art code porting to Python.

In [1]:


import numpy as np

from quaternion import Quaternions
from op_map import formula_to_op_tree, OpDescr, QOP_tree, r_dispatch



In [2]:
# define image shape
height = 300
width = 300
depth = 1
img_shape = (width, height, 1)

In [3]:
formula_str = "( A2 ( mod golden x+iy ) ( /( +( +( A1 j y+k ) y+k ) y+k ) x+iy ) )"

In [4]:
op_tree = formula_to_op_tree(formula_str)

op_tree

(OpDescr(name='A2', degree=2, call_str='qaut2(a, b)', call_name='qaut2', depends_on_coords=False),
 (OpDescr(name='mod', degree=2, call_str='qmod(a, b)', call_name='qmod', depends_on_coords=False),
  OpDescr(name='golden', degree=0, call_str='qc5()', call_name='qc5', depends_on_coords=False),
  OpDescr(name='x_iy', degree=0, call_str='qcxy(x, y, z)', call_name='qcxy', depends_on_coords=True)),
 (OpDescr(name='/', degree=2, call_str='qdiv(a, b)', call_name='qdiv', depends_on_coords=False),
  (OpDescr(name='+', degree=2, call_str='qplus(a, b)', call_name='qplus', depends_on_coords=False),
   (OpDescr(name='+', degree=2, call_str='qplus(a, b)', call_name='qplus', depends_on_coords=False),
    (OpDescr(name='A1', degree=2, call_str='qaut1(a, b)', call_name='qaut1', depends_on_coords=False),
     OpDescr(name='j', degree=0, call_str='qc3()', call_name='qc3', depends_on_coords=False),
     OpDescr(name='y_k', degree=0, call_str='qcy1(x, y, z)', call_name='qcy1', depends_on_coords=True)),
   

In [5]:
op_tree[0]

OpDescr(name='A2', degree=2, call_str='qaut2(a, b)', call_name='qaut2', depends_on_coords=False)

In [6]:
# build image tensors
x_vec = np.linspace(start=0, stop=1, num=width)
y_vec = np.linspace(start=0, stop=1, num=height)
z_vec = np.zeros(1)

x = np.zeros(img_shape)
y = np.zeros(img_shape)
z = np.zeros(img_shape)

for xi in range(width):
    for yi in range(height):
        for zi in range(depth):
            x[xi][yi][zi] = x_vec[xi]
            y[xi][yi][zi] = y_vec[yi]
            z[xi][yi][zi] = z_vec[zi]

In [7]:
res = r_dispatch(op_tree, img_shape=img_shape)

In [8]:
res.q.e

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])

In [9]:
res_q = res.eval_tree(x=x, y=y, z=z)

In [10]:
res_q.e

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [1.5       ],
        [1.2       ],
        ...,
        [0.0101009 ],
        [0.010067  ],
        [0.01003333]],

       [[0.        ],
        [1.2       ],
        [1.5       ],
        ...,
        [0.0202011 ],
        [0.02013332],
        [0.02006599]],

       ...,

       [[0.        ],
        [0.0101009 ],
        [0.0202011 ],
        ...,
        [1.5       ],
        [1.49999153],
        [1.49996622]],

       [[0.        ],
        [0.010067  ],
        [0.02013332],
        ...,
        [1.49999153],
        [1.5       ],
        [1.49999158]],

       [[0.        ],
        [0.01003333],
        [0.02006599],
        ...,
        [1.49996622],
        [1.49999158],
        [1.5       ]]])

In [11]:
res.q.e

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [1.5       ],
        [1.2       ],
        ...,
        [0.0101009 ],
        [0.010067  ],
        [0.01003333]],

       [[0.        ],
        [1.2       ],
        [1.5       ],
        ...,
        [0.0202011 ],
        [0.02013332],
        [0.02006599]],

       ...,

       [[0.        ],
        [0.0101009 ],
        [0.0202011 ],
        ...,
        [1.5       ],
        [1.49999153],
        [1.49996622]],

       [[0.        ],
        [0.010067  ],
        [0.02013332],
        ...,
        [1.49999153],
        [1.5       ],
        [1.49999158]],

       [[0.        ],
        [0.01003333],
        [0.02006599],
        ...,
        [1.49996622],
        [1.49999158],
        [1.5       ]]])